In [ ]:
import os
import json
import asyncio
from pydantic import BaseModel, Field
from functools import partial
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

# from dotenv import load_dotenv
from typing import List, Optional, Callable, Any

# load_dotenv()
api_key = os.getenv('OPEN_ROUTER_API_KEY')

if not api_key:
    raise ValueError("OPEN_ROUTER_API_KEY is missing.")

llm_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/mistral-7b-instruct",
    api_key=api_key,
    model_info={
        "family": 'gemini',
        "vision": True,
        "function_calling": True,
        "json_output": False,
        "structured_output": False
    }
)

class ParsedIdeaModel(BaseModel):
    problem: str
    solution: str
    domain: str

class MarketSizeModel(BaseModel):
    TAM: str
    SAM: str
    SOM: str

class SuggestedStack(BaseModel):
    frontend: str
    backend: str
    database: str
    infrastructure: str
    others: str

class TechFeasibilityModel(BaseModel):
    tech_feasibility: str
    tech_risks: Optional[List[str]] = None
    suggested_stack: Optional[SuggestedStack] = None

class Competitor(BaseModel):
    name: str
    description: str

class CompetitorModel(BaseModel):
    direct_competitors: Optional[List[Competitor]]
    indirect_competitors: Optional[List[Competitor]]
    differentiation_factors: Optional[List[str]]

class GTMStrategyModel(BaseModel):
    target_segments: Optional[List[str]]
    distribution_channels: Optional[List[str]]
    early_adopter_hooks: Optional[List[str]]
    initial_launch_plan: str
    growth_tactics: Optional[List[str]]

class RevenueModel(BaseModel):
    primary_revenue_model: str
    secondary_revenue_model: Optional[str]
    pricing_strategy: str
    value_capture_mechanism: str

class FinanceModel(BaseModel):
    revenue_streams: List[str]
    cost_structure: List[str]
    monthly_burn_rate_estimate: str
    twelve_month_revenue_projection: str
    unit_economics_insight: str

class AgentState(BaseModel):
    raw_idea: str
    parsed_idea_model: ParsedIdeaModel = None
    market_size_model: MarketSizeModel = None
    tech_feasibility_model: TechFeasibilityModel = None
    competitor_model: CompetitorModel = None
    gtm_strategy_model: GTMStrategyModel = None
    revenue_model: RevenueModel = None
    finance_model: FinanceModel = None

state = AgentState(raw_idea="Hi")


#### Prompts

In [ ]:
agent_prompt_idea = """\
You are a startup idea parser agent.

Your task is to read a raw user-submitted startup idea — usually in free-form natural language — and extract the following three key structured fields:

1. **problem** - What issue, pain point, or inefficiency is the idea trying to solve?
2. **solution** - What is the proposed solution, technology, or service offered to address the problem?
3. **domain** - What broad industry, vertical, or category does this idea belong to (e.g., healthcare, education, fintech, logistics)?

Respond ONLY in JSON format using the following schema:

```json
{
  "problem": "<clearly defined problem statement>",
  "solution": "<clear explanation of the proposed solution>",
  "domain": "<relevant domain name>"
}"""

agent_prompt_market_size = """\
You are a market sizing agent for early-stage startups.

Your task is to analyze the provided startup idea and estimate the overall market potential using the standard TAM, SAM, and SOM framework:

1. **TAM (Total Addressable Market)** - The global market demand for this type of product or solution, assuming ideal conditions and full adoption across all segments.
2. **SAM (Serviceable Available Market)** - The portion of TAM that is realistically reachable by the startup based on geography, target customer type, or product scope.
3. **SOM (Serviceable Obtainable Market)** - The subset of SAM that the startup can likely capture in the next 3-5 years, considering existing competitors, available resources, and GTM strategy.

Here is the required information to guide your estimation:
problem: {problem}
solution: {solution}
domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "TAM": "<e.g., $100B - global estimate with logic>",
  "SAM": "<e.g., $15B - narrowed by region/segment with reasoning>",
  "SOM": "<e.g., $300M - estimated achievable share in 3-5 years with justification>"
}}
"""

agent_prompt_tech_feasibility = """\
You are a technical feasibility agent tasked with evaluating the technical implementation potential of a startup idea.

Your job is to analyze the provided startup concept and produce the following structured outputs:

1. **tech_feasibility** - A short paragraph (2-4 sentences) assessing how feasible the idea is from a technical perspective based on current tools, infrastructure, and maturity of required technologies.
2. **tech_risks** - A list of major technical challenges or uncertainties (e.g., scalability, data availability, algorithmic complexity, integration hurdles).
3. **suggested_stack** - A high-level proposed tech stack including backend, frontend, database, infrastructure, and any AI/ML or third-party service components where applicable.

Here is the startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "tech_feasibility": "<short narrative on feasibility>",
  "tech_risks": [
    "<risk 1>",
    "<risk 2>",
    "<risk 3>"
  ],
  "suggested_stack": {{
    "frontend": "<framework or library>",
    "backend": "<framework or language>",
    "database": "<db type>",
    "infrastructure": "<cloud/provider/services>",
    "others": "<any ML/AI/third-party tools if needed>"
  }}
}}
"""

agent_prompt_competitor = """\
You are a competitor landscape analysis agent tasked with identifying the startup's direct and indirect competitors, along with what differentiates it in the market.

Analyze the given startup idea and return:

1. **direct_competitors** – Companies or platforms offering the same or very similar services or products to the same target audience.
2. **indirect_competitors** – Companies solving a similar problem but using a different approach, or targeting a related audience or adjacent market.
3. **differentiation_factors** – 2 to 5 concise, strategic differentiators that explain how this startup stands out from the competition.

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "direct_competitors": [
    {{
      "name": "<Company Name>",
      "description": "<Concise summary of their offering>"
    }}
  ],
  "indirect_competitors": [
    {{
      "name": "<Company Name>",
      "description": "<Concise summary of their offering>"
    }}
  ],
  "differentiation_factors": [
    "<Key differentiator 1>",
    "<Key differentiator 2>",
    "<Key differentiator 3>"
  ]
}}
"""

agent_prompt_gtm_strategy = """\
You are a Go-To-Market (GTM) strategy agent for a startup.

Your role is to analyze a startup's idea and produce a structured GTM plan. Your focus is on identifying the most effective way to launch and grow the product in its target markets.

Analyze the startup context and return the following components:

1. **target_segments** – Who are the ideal first users or customers? Segment by geography, user type, institution type, or behavior.
2. **distribution_channels** – How will the product reach the users? Include digital, partnership-based, and physical channels if relevant.
3. **early_adopter_hooks** – Which features, incentives, or value propositions will attract the first wave of users?
4. **initial_launch_plan** – Briefly outline the go-to-market launch region and phase strategy.
5. **growth_tactics** – Scalable tactics for expansion after launch (e.g., referral systems, ambassador programs, SEO, paid ads, strategic partnerships).

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "target_segments": ["<segment 1>", "<segment 2>"],
  "distribution_channels": ["<channel 1>", "<channel 2>"],
  "early_adopter_hooks": ["<hook 1>", "<hook 2>"],
  "initial_launch_plan": "<concise 2–3 sentence rollout plan>",
  "growth_tactics": ["<growth tactic 1>", "<growth tactic 2>"]
}}
"""

agent_prompt_revenue_model = """\
You are a revenue model agent for an early-stage startup.

Your task is to analyze the startup's offering and propose a monetization strategy that fits the user base, product nature, and domain. Focus on real-world, straightforward revenue mechanisms used by successful SaaS, B2B, or marketplace businesses.

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Return the following fields strictly in valid JSON format:

```json
{{
  "primary_revenue_model": "<Core monetization model, e.g., subscriptions, commissions, usage-based billing>",
  "secondary_revenue_model": "<Optional additional income stream, e.g., ads, affiliate links, white-label licensing>",
  "pricing_strategy": "<Brief pricing approach, e.g., freemium, per-seat, tiered plans>",
  "value_capture_mechanism": "<How the business captures value from usage or transactions>"
}}
"""

agent_prompt_finance = """\
You are a startup financial modeling agent.

Your task is to analyze a startup's business idea and provide an early-stage financial model focused on revenue, burn rate, and basic unit economics. Assume the company is at the pre-seed or seed stage, aiming for product-market fit and fundraising readiness.

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}
- Revenue model: {revenue_model}  (e.g., subscriptions, usage-based billing, ads)

Respond strictly in the following JSON format:

```json
{{
  "revenue_streams": [
    "<Primary revenue stream (e.g., landlord listing fees)>",
    "<Secondary revenue stream (e.g., student premium subscriptions)>"
  ],
  "cost_structure": [
    "<Main fixed or variable costs (e.g., engineering, cloud infra, marketing)>"
  ],
  "monthly_burn_rate_estimate": "<Approximate early-stage monthly burn (e.g., $25K–$50K)>",
  "twelve_month_revenue_projection": "<Expected revenue range (e.g., $100K–$300K)>",
  "unit_economics_insight": "<Short commentary on CAC vs LTV, breakeven point, or payback period>"
}}
"""


#### Agents

In [ ]:
idea_parser_agent = AssistantAgent(name="idea_parser_agent",
                        model_client=llm_client,
                        description="Parses raw, unstructured startup ideas submitted in natural language and extracts three key business components: the core problem being solved, the proposed solution, and the relevant industry domain. The agent outputs a structured JSON object with clearly labeled fields, enabling downstream agents to perform targeted validation (e.g., market size, tech feasibility). Optimized for free-form input from non-technical or early-stage founders.",
                        system_message=agent_prompt_idea)

market_size_agent = AssistantAgent(name="market_size_agent",
                                   description="Estimates the market potential of a startup idea by calculating Total Addressable Market (TAM), Serviceable Available Market (SAM), and Serviceable Obtainable Market (SOM). The agent analyzes the problem, solution, and domain to deliver structured, approximate figures with reasoning. Outputs help determine the scale of opportunity, guide financial projections, and validate investor readiness. Tailored for early-stage startups evaluating market entry feasibility.",
                                   model_client=llm_client,
                                   system_message="You are a market sizing expert. Your role is to estimate TAM, SAM, and SOM based on a startup idea. Respond only with structured JSON outputs.")

tech_feasibility_agent = AssistantAgent(
    name="tech_feasibility_agent",
    model_client=llm_client,
    description="Evaluates the technical feasibility of a startup idea by assessing the implementability of the proposed solution using current tools, infrastructure, and AI/ML capabilities. The agent identifies major technical risks and recommends a high-level technology stack—including backend, frontend, database, infrastructure, and relevant third-party or AI services. Tailored for early-stage startups aiming to validate buildability and derisk engineering execution.",
    system_message="You are a technical feasibility expert. Assess whether the solution is technically viable, list major risks, and suggest a high-level tech stack. Respond only in JSON format with the required fields."
)

competitor_agent = AssistantAgent(name="competitor_agent",
                       model_client=llm_client,
                       description="Performs competitor landscape analysis for a startup idea by identifying direct and indirect competitors and highlighting key differentiation factors. The agent analyzes the problem, solution, and domain to generate a structured comparison against existing players in the market. Designed to support GTM strategy, investor readiness, and positioning decisions for early-stage startups.",
                       system_message="You are a competitor analysis expert. Identify direct and indirect competitors based on the startup idea, and list 2–5 key differentiation factors. Return output strictly in JSON format as specified."
                    )

gtm_strategy_agent = AssistantAgent(
    name="gtm_strategy_agent",
    model_client=llm_client,
    description="Generates a structured Go-To-Market (GTM) strategy for a startup by analyzing its problem, solution, and domain. The agent identifies target customer segments, effective distribution channels, compelling early adopter incentives, and actionable launch and growth tactics. Designed to help early-stage startups build scalable market entry plans and investor-ready GTM narratives.",
    system_message="You are a Go-To-Market strategy expert. Based on the startup idea, return a structured JSON with: target_segments, distribution_channels, early_adopter_hooks, initial_launch_plan, and growth_tactics. Do not include any text outside the JSON object."
)

revenue_agent = AssistantAgent(name="revenue_agent",
                       model_client=llm_client,
                       description="Generates a structured revenue model for a startup by analyzing its problem, solution, and domain. The agent proposes primary and secondary monetization strategies, pricing approach, and value capture mechanisms based on real-world patterns used in SaaS, B2B, or platform-based business models. Designed to support business model validation, pitch readiness, and financial planning for early-stage startups.",
                       system_message="You are a revenue model strategist. Based on the startup idea, return a structured JSON with: primary_revenue_model, secondary_revenue_model, pricing_strategy, and value_capture_mechanism. Do not include any explanation or text outside the JSON object.")

finance_agent = AssistantAgent(name="finance_agent",
                       model_client=llm_client,
                       description="Generates early-stage financial projections for a startup by analyzing its business model, revenue strategy, and domain. The agent estimates revenue streams, cost structure, monthly burn rate, and 12-month revenue projections, while offering insight into unit economics such as CAC vs LTV or breakeven timeline. Optimized for SaaS and tech-driven startups targeting product-market fit and investor-readiness.",
                       system_message="You are a financial modeling assistant for early-stage startups. Based on the startup’s idea and revenue model, return a JSON object with: revenue_streams, cost_structure, monthly_burn_rate_estimate, twelve_month_revenue_projection, and unit_economics_insight. Ensure estimates are realistic and benchmarked against typical SaaS or tech startup metrics. Do not include any explanation or text outside the JSON object.")


#### Idea Parser

In [4]:
async def run_agent_idea_parser(state: AgentState, feedback: str="") -> ParsedIdeaModel:

    if not feedback:
        task = state.raw_idea
    else:
        task = state.raw_idea + f"\n\nNOTE: User feedback - {feedback}"

    result = await idea_parser_agent.run(task=task)

    idea_txt = result.messages[-1].content

    if idea_txt:

        if "```json" in idea_txt:
            idea_txt = idea_txt.replace("```json", "").replace("```", "").strip()

        idea_json = json.loads(idea_txt)
        parsed_model = ParsedIdeaModel(**idea_json)

        return parsed_model
    
    return ParsedIdeaModel(problem='', solution='', domain='')

#### Market

In [5]:
async def run_agent_marketsize(state: AgentState, feedback: str="") -> MarketSizeModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:
        
        prompt = agent_prompt_market_size.format(problem=state.parsed_idea_model.problem, 
                                                                solution=state.parsed_idea_model.solution, 
                                                                domain=state.parsed_idea_model.domain)
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"
            
        result = await market_size_agent.run(task=prompt)

        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            parsed_market_size = MarketSizeModel(**result_json)

            return parsed_market_size

    return MarketSizeModel(TAM='', SAM='', SOM='')

#### Tech Feasibility

In [6]:
async def run_agent_tech_feasibility(state: AgentState, feedback: str="") -> TechFeasibilityModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_tech_feasibility.format(
        problem=state.parsed_idea_model.problem,
        solution=state.parsed_idea_model.solution,
        domain=state.parsed_idea_model.domain)
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"
        
        result = await tech_feasibility_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            tech_feasibility_result = TechFeasibilityModel(**result_json)

            return tech_feasibility_result
        
    return TechFeasibilityModel(
    tech_feasibility="",
    tech_risks=[],
    suggested_stack=SuggestedStack(
        frontend="",
        backend="",
        database="",
        infrastructure="",
        others=""
    )
)

#### Competitor

In [7]:
async def run_agent_competitor(state: AgentState, feedback: str='') -> CompetitorModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_competitor.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain
            )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await competitor_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            compitator_result = CompetitorModel(**result_json)

            return compitator_result
        
    return CompetitorModel(
        direct_competitors=[Competitor(name='', description='')],
        indirect_competitors=[Competitor(name='', description='')],
        differentiation_factors=[]
    )


#### GTM Strategy

In [8]:
async def run_agent_gtm_strategy(state: AgentState, feedback: str='') -> GTMStrategyModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_gtm_strategy.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain
            )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await gtm_strategy_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            gtm_strategy_result = GTMStrategyModel(**result_json)

            return gtm_strategy_result
        
    return GTMStrategyModel(
    target_segments=[],
    distribution_channels=[],
    early_adopter_hooks=[],
    initial_launch_plan="",
    growth_tactics=[]
)

#### Revenue

In [9]:
async def run_agent_revenue(state: AgentState, feedback: str='') -> RevenueModel:

    if(
        state.parsed_idea_model.problem and 
        state.parsed_idea_model.solution and 
        state.parsed_idea_model.domain and 
        state.revenue_model
    ):
        prompt = agent_prompt_finance.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain,
            revenue_model=state.revenue_model.model_dump_json(indent=2)
        )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await revenue_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            revenue_result = RevenueModel(**result_json)

            return revenue_result
        
    return RevenueModel(primary_revenue_model='', secondary_revenue_model='', pricing_strategy='', value_capture_mechanism='')

#### Finance

In [10]:
async def run_agent_finance(state: AgentState, feedback: str='') -> FinanceModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_finance.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain,
            revenue_model=state.revenue_model.model_dump_json(indent=2)
            )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await finance_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            finance_result = FinanceModel(**result_json)

            return finance_result
        
    return FinanceModel(revenue_streams=[], cost_structure=[], monthly_burn_rate_estimate='', twelve_month_revenue_projection='', unit_economics_insight='')

#### Human in the loop

In [11]:
async def hitl_feedback_loop(output_name: str, initial_output: Any, rerun_fn: Callable[[str], Any]):
    attempt = 1
    max_attempt = 3
    current_output = initial_output

    while max_attempt >= attempt:

        print(f"\nOutput from {output_name} (Attempt #{attempt}):")
        print(json.dumps(current_output.model_dump(), indent=2))

        decision = input(f"\nApprove output from {output_name} (y/n)?").strip().lower()

        if decision == 'y':
            print(f"Approved: {current_output}")
            return current_output
        
        print("retrying...")
        feedback = input("Enter feedback to improve: ")
        current_output = await rerun_fn(feedback)

        attempt += 1

    print(f"max attempts reached for {output_name}, returning last output.")
    return current_output

#### Update Agent State

In [12]:
def update_agent_state(state: AgentState, **kwargs: Any) -> AgentState:
    """
    Returns a new AgentState with updated fields.
    
    Args:
        state: The current AgentState instance
        **kwargs: Fields to update in the state (e.g., parsed_idea_model=..., revenue_model=...)
    
    Returns:
        A new AgentState instance with updated fields
    """
    return state.model_copy(update=kwargs)

In [30]:
# startup_idea = input("Enter the startup idea: ").strip()

# if startup_idea:

#     state = update_agent_state(state, raw_idea=startup_idea)
#     result = await run_agent_idea_parser(state)
#     rerun_fn = partial(run_agent_idea_parser, state)
#     idea_result = await hitl_feedback_loop('Idea parser', result, rerun_fn)
#     state = update_agent_state(state, parsed_idea_model=idea_result)

#     result = await run_agent_marketsize(state)
#     rerun_fn = partial(run_agent_marketsize, state)
#     market_result = await hitl_feedback_loop('market research', result, rerun_fn)
#     state = update_agent_state(state, market_size_model=market_result)
    
#     result = await run_agent_tech_feasibility(state)
#     rerun_fn = partial(run_agent_tech_feasibility, state)
#     tech_feasibility_result = await hitl_feedback_loop('Tech feasibility', result, rerun_fn)
#     state = update_agent_state(state, tech_feasibility_model=tech_feasibility_result)
    
#     result = await run_agent_competitor(state)
#     rerun_fn = partial(run_agent_competitor, state)
#     compitator_result = await hitl_feedback_loop('Compitator', result, rerun_fn)
#     state = update_agent_state(state, competitor_model=compitator_result)

#     result = await run_agent_gtm_strategy(state)
#     rerun_fn = partial(run_agent_gtm_strategy, state)
#     gtm_strategy_result = await hitl_feedback_loop('GTM Strategy', result, rerun_fn)
#     state = update_agent_state(state, gtm_strategy_model=gtm_strategy_result)

#     result = await run_agent_revenue(state)
#     rerun_fn = partial(run_agent_revenue, state)
#     revenue_result = await hitl_feedback_loop('Revenue', result, rerun_fn)
#     state = update_agent_state(state, revenue_model=revenue_result)
    
#     result = await run_agent_finance(state)
#     rerun_fn = partial(run_agent_finance, state)
#     finance_result = await hitl_feedback_loop('Finance', result, rerun_fn)
#     state = update_agent_state(state, finance_model=finance_result)

#     print(f"Final:\n{state.model_dump_json(indent=2)}")



#### Orchestration